In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
season = "2023-2024"

In [47]:
url = "https://fbref.com/en/comps/11/2023-2024/keepers/2023-2024-Serie-A-Stats"

In [63]:
resp = requests.get(url)
resp.encoding = 'utf-8'
soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')

headers, rows = [], []

table_data = soup.find_all('tbody')[2]
table_rows = table_data.find_all('tr')

for value in table_rows[0]:
    headers.append(value.get('data-stat'))

for row in table_rows:
    row_values = []
    row_data = row.find_all('td')
    for data_value in row_data:
        if data_value.get('data-append-csv'):
            row_values.append(data_value.get('data-append-csv'))
        row_values.append(data_value.get_text())
    rows.append(row_values)

df = pd.DataFrame(rows, columns = headers).fillna(0.0)
df = df.loc[df['player'] != 0].reset_index(drop = True)

df['gk_minutes'] = df['gk_minutes'].str.replace(',', '')

for i, row in df.iterrows():
    try:
        if '-' in row['age']:
            age_num = row['age'].split('-')[0]
            df.at[i, 'age'] = age_num
    except: pass

df.rename(columns={'ranker' : 'player_id'}, inplace=True)
df = df.apply(pd.to_numeric, errors = 'ignore')

In [65]:
df.dtypes

player_id                      object
player                         object
nationality                    object
position                       object
team                           object
age                             int64
birth_year                      int64
gk_games                        int64
gk_games_starts                 int64
gk_minutes                      int64
minutes_90s                   float64
gk_goals_against                int64
gk_goals_against_per90        float64
gk_shots_on_target_against      int64
gk_saves                        int64
gk_save_pct                   float64
gk_wins                         int64
gk_ties                         int64
gk_losses                       int64
gk_clean_sheets                 int64
gk_clean_sheets_pct           float64
gk_pens_att                     int64
gk_pens_allowed                 int64
gk_pens_saved                   int64
gk_pens_missed                  int64
gk_pens_save_pct              float64
matches     